In [23]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor

In [2]:
file_train = Path('../data/Grupo_Bimbo_Inventory_Demand/train.csv')
file_test = Path('../data/Grupo_Bimbo_Inventory_Demand/test.csv')

In [3]:
df_train = pd.read_csv(file_train)
df_test = pd.read_csv(file_test)

In [4]:
df_train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.00,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.00,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.00,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.00,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.00,3
...,...,...,...,...,...,...,...,...,...,...,...
74180459,9,25759,1,5517,4528866,32873,4,27.76,0,0.00,4
74180460,9,25759,1,5517,4528866,34226,4,37.04,0,0.00,4
74180461,9,25759,1,5517,4528866,45112,4,51.00,0,0.00,4
74180462,9,25759,1,5517,4547943,40217,3,26.94,3,26.94,0


In [5]:
cols_train = list(df_train.columns)
cols_test = list(df_test.columns)

In [6]:
cols_train = set(cols_train)
cols_test = set(cols_test)

In [7]:
cols = cols_test.intersection(cols_train)

In [8]:
df_train_raw = df_train.loc[:, list(cols)+['Demanda_uni_equil']]

In [9]:
df_train_raw

,Cliente_ID,Agencia_ID,Canal_ID,Producto_ID,Ruta_SAK,Semana,Demanda_uni_equil
0,15766,1110,7,1212,3301,3,3
1,15766,1110,7,1216,3301,3,4
2,15766,1110,7,1238,3301,3,4
3,15766,1110,7,1240,3301,3,4
4,15766,1110,7,1242,3301,3,3
...,...,...,...,...,...,...,...
74180459,4528866,25759,1,32873,5517,9,4
74180460,4528866,25759,1,34226,5517,9,4
74180461,4528866,25759,1,45112,5517,9,4
74180462,4547943,25759,1,40217,5517,9,0


In [10]:
X = df_train_raw.drop("Demanda_uni_equil", axis=1)
y = df_train_raw.Demanda_uni_equil

In [11]:
X = pd.DataFrame(minmax_scale(X), columns=X.columns)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.4,
    random_state=42,
    shuffle=False
)

In [17]:
sgd_reg = SGDRegressor(
    early_stopping=True,
    max_iter=10_000,
    shuffle=False,
    verbose=2,
    random_state=42
)

In [18]:
sgd_reg.fit(X_train, y_train)

-- Epoch 1
Norm: 43.44, NNZs: 6, Bias: 14.001698, T: 40057450, Avg. loss: 196.252815
Total training time: 2.80 seconds.
-- Epoch 2
Norm: 47.60, NNZs: 6, Bias: 19.574933, T: 80114900, Avg. loss: 198.314214
Total training time: 5.76 seconds.
-- Epoch 3
Norm: 49.21, NNZs: 6, Bias: 22.850320, T: 120172350, Avg. loss: 199.053467
Total training time: 8.69 seconds.
-- Epoch 4
Norm: 50.17, NNZs: 6, Bias: 24.774873, T: 160229800, Avg. loss: 199.517779
Total training time: 11.89 seconds.
-- Epoch 5
Norm: 50.73, NNZs: 6, Bias: 25.940783, T: 200287250, Avg. loss: 199.856323
Total training time: 14.92 seconds.
-- Epoch 6
Norm: 51.03, NNZs: 6, Bias: 26.668135, T: 240344700, Avg. loss: 200.121857
Total training time: 17.98 seconds.
Convergence after 6 epochs took 18.16 seconds


SGDRegressor(early_stopping=True, max_iter=10000, random_state=42,
             shuffle=False, verbose=2)

In [24]:
np.array(sgd_reg.predict(X_test))

array([  5.45741854,   5.45887195,   5.45426943, ..., -41.55468017,
       -42.14757574, -41.79147243])

In [26]:
np.array(y_test)

array([4, 7, 4, ..., 4, 0, 1], dtype=int64)